<a href="https://colab.research.google.com/github/JoseFerrer/Intro_Signal_Proc/blob/main/Simulaci%C3%B3n_de_fatiga_muscular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simulación de fatiga muscular**


Este notebook usa [PyMuscle](https://github.com/iandanforth/pymuscle) una librería escrita en Python y que está basada en el paper  [**A motor unit-based model of muscle fatigue** de Potvin & Fuglevand (2017)](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005581). PyMuscle se enfoca en la relación entra las señales de control (estimulación de las entradas a las neuronas motoras) y la salida por unidad motora, sin embargo, la simulación sucede para una población de unidades motoras.


**¿Qué es la fatiga muscular?**

Es el declive temporal de la fuerza y capacidad de los músculos esqueléticos resultando en la disminución de actividad muscular.

**¿Qué aplicaciones tiene un modelo y el estudio de la fatiga muscular?**

- Ergonomía
- Rehabilitación
- Ejercicio y deporte

**¿En qué condiciones se realiza la simulación?**

Solamente se simula la fatiga de las unidades motoras asociadas a contracciones 
- Sostenidas
- Isométrias

**¿Qué preguntas puede responder el modelo de simulación?**

- Cómo varía la contribución de las fuerzas de unidades motoras individuales a través del tiempo
- Qué subpoblaciones de unidades motoras soportan los mayores grados de fatiga para diferentes tipos de contracciones
- Para tareas que alcanzan el mismo nivel de pérdida de la fuerza muscular, ¿las unidades motoras individuales se fatigan en el mismo grado?


### **Instalación de la librería PyMuscle**

In [ ]:
!pip install pymuscle

### **Importación de las librerías**

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from pymuscle import PotvinFuglevandMuscle as Muscle
from pymuscle.vis import PotvinChart
import plotly.graph_objects as go
import matplotlib.pyplot as plt


In [ ]:
#@title Función para mostrar el gráfico de fuerzas
def display(self):
    # Per Motor Unit Force
    data = []
    for i, t in enumerate(self._forces_by_time):
        trace = dict(
            x=self._times,
            y=t,
            name=i + 1,
            marker=dict(
                color=self._get_color(i)
            ),
        )
        data.append(trace)

    layout = dict(
        title='Fuerza de las unidades motoras por Tiempo',
        yaxis=dict(
            title='Fuerza de las unidades motoras'
        ),
        xaxis=dict(
            title='Tiempo (s)'
        )
    )

    fig = dict(
        data=data,
        layout=layout
    )
    return fig

## **Parámetros de la simulación**

### **Músculo a simular**

Vamos a simular el [el primer músculo interóseo dorsal de la mano](https://es.wikipedia.org/wiki/M%C3%BAsculos_inter%C3%B3seos_dorsales_de_la_mano)

Y se puede observar a continuación

![](https://upload.wikimedia.org/wikipedia/commons/5/57/1121_Intrinsic_Muscles_of_the_Hand_DIL.png)



### **Número de unidades motoras**





In [ ]:
motor_unit_count = 119 
muscle = Muscle(motor_unit_count)

De acuerdo a al paper **[Morphologic studies of motor units in normal human muscles](https://drive.google.com/open?id=1XZLDO-2itC_7stQucxklM4UJ7UXy3f8K)** de Feinstein, B., Lindegård, B., Nyman, E., & Wohlfart, G. (1955) publicado en  Cells Tissues Organs, 23(2), 127-142 el primer músculo interóseo dorsal de la mano tiene 119 unidades motoras.

![](https://drive.google.com/uc?id=1zKvjQ3tjJFJeab18tD3WZrfYdUbtupg-)

### **Fuerza máxima voluntaria**



La fuera máxima voluntaria del primer músculo interóseo dorsal de la mano se estima en 32 ± 12 Newtons (rango: 17–55 N)  de acuerdo al paper **[EMG-force relation in the first dorsal interosseous muscle of patients with amyotrophic lateral sclerosis](https://www.ncbi.nlm.nih.gov/pubmed/24990032)** de Jahanmiri-Nezhad, F., Hu, X., Suresh, N. L., Rymer, W. Z., & Zhou, P. (2014) publicado en NeuroRehabilitation, 35(2), 307-314.

### **Otros parámetros de la simulación**

- Con pocas excepciones, las unidades motoras parecen ser reclutadas en una secuencia ordenada, comenzando con los que ejercen las menores fuerzas hasta lo que ejercen las mayores

- El mecanismo neuronal subyacente al reclutamiento ordenado de las unidades motoras (de débiles y menos fatigables a fuertes y más fatigables) han sido estudiados por Hanneman y conocido como "size principle". Puede ver más [aquí](https://journals.physiology.org/doi/pdf/10.1152/classicessays.00025.2005)

- Una vez reclutada una unidad motoral incrementa su firing rate con una excitación sináptica incrementada en un rango limitado de valores antes de saturarse a niveles que parecieran inversamente relacionados al threshold de reclutamiento de las unidades motoras.


In [ ]:
#@title Estímulo { run: "auto" }
#Usar un nivel constante de estímulo para observar la fatiga
#@markdown A continuación puede cambiar el 
#@markdown nivel de esfuerzo aplicado al músculo
excitation = 20 #@param {type:"slider", min:0, max:100, step:1}
tiempo_a_simular = 200 #@param ["200"] {type:"raw", allow-input: false}
frames_per_second = 50 #@param ["50"] {type:"raw", allow-input: false}
step_size = 1 / frames_per_second
total_steps = int(tiempo_a_simular / step_size)
total_outputs = []
outputs_by_unit = []
print("Empezando simulación...")
for i in tqdm(range(total_steps)):
    # Calling step() updates the simulation and returns the total output
    # produced by the muscle during this step for the given excitation level.
    total_output = muscle.step(excitation, step_size)
    total_outputs.append(total_output)
    # You can also introspect the muscle to see the forces being produced
    # by each motor unit.
    output_by_unit = muscle.current_forces
    outputs_by_unit.append(output_by_unit)
print("Simulación ejecutada para {} segundos ...".format(tiempo_a_simular))
# Visualizar el comporamiento de las unidades motoras a través del tiempo
print("Creando gráfico...")
chart = PotvinChart(outputs_by_unit,step_size)
fig = go.Figure( **display(chart) )
fig.show()  

# **Práctica Dirigida**

## **Ejercicio 01 (10 puntos)**
A diferencia del código anterior en el que el esfuerzo es constante, genere una función de esfuerzo variable. **Nota:** Recuerde que las señales deben ser totalmente positivas y tener como valor máximo `excitation`. Se provee como ejemplo la función constante.

In [ ]:
#@title Estímulo { run: "auto" }
#Usar un nivel constante de estímulo para observar la fatiga
#@markdown A continuación puede cambiar el 
#@markdown nivel de esfuerzo aplicado al músculo
excitation = 75 #@param {type:"slider", min:0, max:100, step:1}
tipo_onda = "constant" #@param ["constant", "square", "sine"]
tiempo_a_simular = 200 #@param ["200"] {type:"raw", allow-input: false}
frames_per_second = 50 #@param ["50"] {type:"raw", allow-input: false}
seconds_per_period = 34 #@param {type:"slider", min:1, max:200, step:1}
 

##### Desarrolle su código a partir de aquí ######
time = np.linspace( 0, tiempo_a_simular, frames_per_second*tiempo_a_simular)

if tipo_onda == 'constant':
  signal = time*0+excitation
elif tipo_onda == 'square':
  ##### Desarrolle su código aquí
  #signal = 

  print("Falta Implementar") #puede eliminar
elif tipo_onda == 'sine':
  ##### Desarrolle su código aquí
  #signal =   
  
  print("Falta Implementar") #puede eliminar

#Plot
plt.figure(figsize=(20,5))
plt.title("Excitation x Tiempo")
plt.xlabel("Tiempo (s)")
plt.ylabel("Excitation")
plt.plot(time, signal)

## **Ejercicio 02 (10 puntos)**

Simule e indique qué tipo de esfuerzo fatiga los músculos (I) más rápido, (II) más lento. Explique cuales son las implicancias

In [ ]:
step_size = 1 / frames_per_second
total_steps = int(tiempo_a_simular / step_size)
total_outputs = []
outputs_by_unit = []
print("Empezando simulación...")
for i in tqdm(range(total_steps)):
    # Calling step() updates the simulation and returns the total output
    # produced by the muscle during this step for the given excitation level.
    total_output = muscle.step(signal[i], step_size)
    total_outputs.append(total_output)
    # You can also introspect the muscle to see the forces being produced
    # by each motor unit.
    output_by_unit = muscle.current_forces
    outputs_by_unit.append(output_by_unit)
print("Simulación ejecutada para {} segundos ...".format(tiempo_a_simular))
# Visualizar el comporamiento de las unidades motoras a través del tiempo
print("Creando gráfico...")
chart = PotvinChart(outputs_by_unit,step_size)
fig = go.Figure( **display(chart) )
fig.show()  